In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
## Data Analysis Phase
## Main aim is to understand more about the data

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
%matplotlib inline
import warnings
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator



warnings.filterwarnings('ignore')


## Display all the columns of the dataframe
pd.pandas.set_option('display.max_columns',None)

In [2]:
dataset = pd.read_csv('/content/drive/MyDrive/Data/hmnist_28_28_RGB.csv')

In [3]:
IMG_H=64
IMG_W=64

In [4]:
classes={0:('actinic keratoses and intraepithelial carcinomae(Cancer)'),
         1:('basal cell carcinoma(Cancer)'),
         2:('benign keratosis-like lesions(Non-Cancerous)'),
         3:('dermatofibroma(Non-Cancerous)'),
         4:('melanocytic nevi(Non-Cancerous)'),
         5:('pyogenic granulomas and hemorrhage(Can lead to cancer)'),
         6:('melanoma(Cancer)')}

In [6]:
augment_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [7]:
# Function to augment images for a specific class
def augment_class_images(df, class_label, target_size, augment_gen):
    class_images = df[df['label'] == class_label].drop(columns=['label']).values
    num_to_augment = target_size - len(class_images)

    augmented_images = []
    for i in range(num_to_augment):
        img = class_images[i % len(class_images)].reshape(28, 28, 3).astype(np.uint8)
        img = augment_gen.random_transform(img)
        augmented_images.append(img.flatten())

    return np.array(augmented_images)

# Define target size for balancing
target_size = 3500

# Augment images for minority classes
augmented_images = []
augmented_labels = []

for class_label in classes.keys():
    if len(dataset[dataset['label'] == class_label]) < target_size:
        aug_images = augment_class_images(dataset, class_label, target_size, augment_gen)
        if aug_images.size > 0:
            augmented_images.append(aug_images)
            augmented_labels.extend([class_label] * len(aug_images))

# Combine original and augmented data
augmented_images = np.vstack(augmented_images)
augmented_labels = np.array(augmented_labels)

# Create a new dataframe with augmented data
augmented_df = pd.DataFrame(augmented_images, columns=dataset.columns[:-1])
augmented_df['label'] = augmented_labels

# Combine with the original dataset
balanced_dataset = pd.concat([dataset, augmented_df], ignore_index=True)

print(balanced_dataset['label'].value_counts())


label
4    6705
2    3500
3    3500
6    3500
5    3500
1    3500
0    3500
Name: count, dtype: int64


In [8]:
label_4_data = balanced_dataset[balanced_dataset['label'] == 4]
num_to_remove = len(label_4_data) - 3500

rows_to_remove = label_4_data.sample(n=num_to_remove, random_state=42).index
balanced_dataset = balanced_dataset.drop(index=rows_to_remove)


print(balanced_dataset['label'].value_counts())# Verifying the change

label
2    3500
4    3500
3    3500
6    3500
5    3500
1    3500
0    3500
Name: count, dtype: int64


In [9]:
balanced_dataset.head()

pixel0000  pixel0001  pixel0002  pixel0003  pixel0004  pixel0005  \
0        192        153        193        195        155        192   
1         25         14         30         68         48         75   
2        192        138        153        200        145        163   
3         38         19         30         95         59         72   
4        158        113        139        194        144        174   

   pixel0006  pixel0007  pixel0008  pixel0009  pixel0010  pixel0011  \
0        197        154        185        202        162        192   
1        123         93        126        158        128        158   
2        201        142        160        206        149        165   
3        143        103        119        171        125        134   
4        215        162        191        225        179        214   

   pixel0012  pixel0013  pixel0014  pixel0015  pixel0016  pixel0017  \
0        208        165        201        209        165        205   
1        172        136        162        175        134        157   
2        207        149        159        209        156        164   
3        177        118        123        190        137        146   
4        232        189        222        235        193        219   

   pixel0018  pixel0019  pixel0020  pixel0021  pixel0022  pixel0023  \
0        208        165        194        212        169        198   
1        177        126        158        186        138        163   
2        202        136        130        200        133        129   
3        192        127        143        200        141        162   
4        234        191        206        238        200        219   

   pixel0024  pixel0025  pixel0026  pixel0027  pixel0028  pixel0029  \
0        216        181        204        216        178        204   
1        197        153        187        198        153        186   
2        208        146        157        212        159        167   
3        207        156        170        202        144        151   
4        244        213        240        243        213        239   

   pixel0030  pixel0031  pixel0032  pixel0033  pixel0034  pixel0035  \
0        215        172        205        217        173        209   
1        203        168        206        205        169        204   
2        214        157        168        211        143        150   
3        202        138        155        212        159        177   
4        242        210        238        244        216        241   

   pixel0036  pixel0037  pixel0038  pixel0039  pixel0040  pixel0041  \
0        217        166        202        214        161        202   
1        207        164        199        205        162        200   
2        215        148        169        213        150        176   
3        211        155        172        211        149        163   
4        247        224        244        246        221        242   

   pixel0042  pixel0043  pixel0044  pixel0045  pixel0046  pixel0047  \
0        217        171        207        218        172        205   
1        206        166        185        209        163        189   
2        203        138        150        194        121        124   
3        203        136        144        204        137        156   
4        246        222        243        245        221        242   

   pixel0048  pixel0049  pixel0050  pixel0051  pixel0052  pixel0053  \
0        215        164        193        211        163        189   
1        208        165        190        203        156        183   
2        195        122        125        189        114        118   
3        209        149        180        206        144        165   
4        249        235        253        249        233        247   

   pixel0054  pixel0055  pixel0056  pixel0057  pixel0058  pixel0059  \
0        204        154        176        204        153        181   
1        198      

In [10]:
file_path = '/content/drive/My Drive/Balanced/balanced_dataset.csv'
balanced_dataset.to_csv(file_path, index=False)

In [11]:
import gc
del dataset
gc.collect()

4

In [12]:
dataset=balanced_dataset

In [13]:
del balanced_dataset
gc.collect()

0

In [14]:
dataset.head()

pixel0000  pixel0001  pixel0002  pixel0003  pixel0004  pixel0005  \
0        192        153        193        195        155        192   
1         25         14         30         68         48         75   
2        192        138        153        200        145        163   
3         38         19         30         95         59         72   
4        158        113        139        194        144        174   

   pixel0006  pixel0007  pixel0008  pixel0009  pixel0010  pixel0011  \
0        197        154        185        202        162        192   
1        123         93        126        158        128        158   
2        201        142        160        206        149        165   
3        143        103        119        171        125        134   
4        215        162        191        225        179        214   

   pixel0012  pixel0013  pixel0014  pixel0015  pixel0016  pixel0017  \
0        208        165        201        209        165        205   
1        172        136        162        175        134        157   
2        207        149        159        209        156        164   
3        177        118        123        190        137        146   
4        232        189        222        235        193        219   

   pixel0018  pixel0019  pixel0020  pixel0021  pixel0022  pixel0023  \
0        208        165        194        212        169        198   
1        177        126        158        186        138        163   
2        202        136        130        200        133        129   
3        192        127        143        200        141        162   
4        234        191        206        238        200        219   

   pixel0024  pixel0025  pixel0026  pixel0027  pixel0028  pixel0029  \
0        216        181        204        216        178        204   
1        197        153        187        198        153        186   
2        208        146        157        212        159        167   
3        207        156        170        202        144        151   
4        244        213        240        243        213        239   

   pixel0030  pixel0031  pixel0032  pixel0033  pixel0034  pixel0035  \
0        215        172        205        217        173        209   
1        203        168        206        205        169        204   
2        214        157        168        211        143        150   
3        202        138        155        212        159        177   
4        242        210        238        244        216        241   

   pixel0036  pixel0037  pixel0038  pixel0039  pixel0040  pixel0041  \
0        217        166        202        214        161        202   
1        207        164        199        205        162        200   
2        215        148        169        213        150        176   
3        211        155        172        211        149        163   
4        247        224        244        246        221        242   

   pixel0042  pixel0043  pixel0044  pixel0045  pixel0046  pixel0047  \
0        217        171        207        218        172        205   
1        206        166        185        209        163        189   
2        203        138        150        194        121        124   
3        203        136        144        204        137        156   
4        246        222        243        245        221        242   

   pixel0048  pixel0049  pixel0050  pixel0051  pixel0052  pixel0053  \
0        215        164        193        211        163        189   
1        208        165        190        203        156        183   
2        195        122        125        189        114        118   
3        209        149        180        206        144        165   
4        249        235        253        249        233        247   

   pixel0054  pixel0055  pixel0056  pixel0057  pixel0058  pixel0059  \
0        204        154        176        204        153        181   
1        198      

Converting 28x28x3 to 64x64x3

In [15]:
import tensorflow as tf
import numpy as np
import pandas as pd

def resize_image(image):
    image = tf.image.resize(image, [IMG_H, IMG_W])
    return image

# Assuming 'dataset' is your loaded DataFrame
images = dataset.iloc[:, :-1].values.reshape(-1, 28, 28, 3).astype(np.uint8)
labels = dataset['label'].values

# Resize images using TensorFlow with tf.data.Dataset for better performance:
def resize_images_with_tf_data(images):
  dataset = tf.data.Dataset.from_tensor_slices(images)
  dataset = dataset.map(resize_image)  # Apply resize_image function to each image
  return np.array(list(dataset.as_numpy_iterator()))

resized_images = resize_images_with_tf_data(images)

# Reshape the resized images and create a DataFrame:
resized_images = resized_images.reshape(resized_images.shape[0], -1)  # Flatten each image
resized_df = pd.DataFrame(resized_images)
dataset = pd.concat([resized_df, pd.Series(labels, name='label')], axis=1)

In [16]:
dataset.tail()

Output hidden; open in https://colab.research.google.com to view.

In [17]:
file_path = '/content/drive/My Drive/Balanced/balanced_resized_dataset_64x64.csv'
dataset.to_csv(file_path, index=False)
